# E-04 인공지능으로 노래 가사 만들기

## 2. 데이터 읽어오기 (시작 지점)

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## 3. 데이터 정제하기

In [2]:
# 문장의 끝이 ':'로 끝나면 대사일 가능성은 적으니 ':'를 기준으로 문장 제외하기.
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue # 길이가 0인 문장은 건너뜀
    if sentence[-1] == ":": continue # 문장 끝이 :인 문장은 건너뜀
    
    if idx > 9: break  ## 작동 시험. 학습 땐 없애야 함.
        
    print(sentence)
        

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [3]:
import re  

def preprocess_sentence(sentence):
    #1. 소문자 바꾸고 양쪽 공백지움.
    sentence = sentence.lower().strip() 
    
    #2. 특수문자 양쪽에 공백.
    sentence = re.sub(r"([?.!,¿])", r"\1 ", sentence) 
    
    #3. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿈.
    sentence = re.sub(r"[^a-zA-Z?.!,¿']+", " ", sentence)
    
    #4. 여러 개의 공백은 하나의 공백으로 바꿈
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    #5. .나 !나?가 등장할 경우 문장 종료. 
    sentence = sentence.replace(".",".<end>")
    sentence = sentence.replace("?","?<end>")
    sentence = sentence.replace("!","!<end>")  
          
    #6. 반복 후렴구문 삭제.  
    sentence = re.sub(r'chorus', " ", sentence)
    sentence = re.sub(r'repeat', " ", sentence)
    sentence = re.sub(r'tag', " ", sentence)
    
    #Last-1. 다시 양쪽 공백을 지움
    sentence = sentence.strip()
    
    #Last. 문장 시작에는 <start>, 끝에는 <end>를 추가.
    sentence = '<start> ' + sentence + ' <end>'
    
    return sentence

print(preprocess_sentence("    This @_is :::sample          sentence.   "))
print(preprocess_sentence("baby, can't you read the signs? i won't bore you with the details, baby "))
print(preprocess_sentence('Niggas see me like, "What up, Killa?"'))
print(preprocess_sentence('When no one asked you. [Chorus: x2] What are we doing,'))
print(preprocess_sentence("Then I pray you're new love's lasting. Repeat Chorus Tag: When love grows cold, it's tragic. "))


<start> this is sample sentence.<end> <end>
<start> baby, can't you read the signs?<end> i won't bore you with the details, baby <end>
<start> niggas see me like, what up, killa?<end> <end>
<start> when no one asked you.<end>   x what are we doing, <end>
<start> then i pray you're new love's lasting.<end>       when love grows cold, it's tragic.<end> <end>


In [4]:
pre_corpus = []  # 정제된 문장 모으기

for sentence in raw_corpus:
    # 원하지 않는 문장 스킵.
    if len(sentence) ==0: continue
    if sentence[-1] ==":": continue # 마지막이 :인것.
        
    # 정제를 하고 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    pre_corpus.append(preprocessed_sentence)
    
# 결과 예시 10개 확인
pre_corpus[40:50]

['<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah suzanne takes you down to her place near the river <end>',
 '<start> you can hear the boats go by, you can spend the night forever <end>',
 "<start> and you know that she's half crazy but that's why you want to be there <end>",
 '<start> and she feeds you tea and oranges that come all the way from china <end>']

In [5]:
#corpus = list(set(pre_corpus)) # 반복구문 삭제.
corpus = pre_corpus  ## 반복 구문 삭제했다가 원복. 반복을 허용한 후 손실 약 0.4~5 감소.
corpus[40:50]

['<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah suzanne takes you down to her place near the river <end>',
 '<start> you can hear the boats go by, you can spend the night forever <end>',
 "<start> and you know that she's half crazy but that's why you want to be there <end>",
 '<start> and she feeds you tea and oranges that come all the way from china <end>']

In [6]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequence를 사용함.
import tensorflow as tf
import numpy as np

def tokenize(corpus):
    # 12000 단어를 기억할 수 있는 tokenizer. 미포함 단어는 '<unk>'
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words = 12000,
    filters = ' ',
    oov_token = "<unk>"
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성.
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환함.
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줌.
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding = 'pre'를 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print("tensor")




[[   2   47  142 ...    0    0    0]
 [   2   14 3939 ...    0    0    0]
 [   2   28    6 ...    0    0    0]
 ...
 [   2  145   18 ...    0    0    0]
 [   2   22   78 ...    0    0    0]
 [   2    6  373 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa4ccf7a4c0>
tensor


In [7]:
print("텐서 크기 변경 - 변경 전")
print(tensor.shape) 

tensor = tensor[:, :15] # 토큰 열 수를 15개로 제한시키기.
print("텐서 크기 변경 - 변경 후. 열 개수 15개로 제한")
print(tensor.shape) # 변경 후 결과 확인

print("15번째 열의 상태 - 변경 전")
print(tensor[:,14]) # 마지막 열만 체크하여 잘리는 부분 확인. 잘리는 부분은 0이나 3이 아닌 숫자가 나올 것임.

tensor[:,14] = np.where(tensor[:,14]>0, 3, 0) # 문장이 중간에서 잘리면 마지막 단어는 종결문자(숫자: 3)로 바꾸기.
print("15번째 열의 상태 - 변경 후. 잘린 문장의 마지막 단어는 종결문자로 치환한다.")
print(tensor[:,14]) # 마지막 단어의 숫자는 0 아니면 3뿐.


텐서 크기 변경 - 변경 전
(175749, 312)
텐서 크기 변경 - 변경 후. 열 개수 15개로 제한
(175749, 15)
15번째 열의 상태 - 변경 전
[   0    0    0 ...    0 2131    0]
15번째 열의 상태 - 변경 후. 잘린 문장의 마지막 단어는 종결문자로 치환한다.
[0 0 0 ... 0 3 0]


In [8]:
import pandas as pd
df_tensor = pd.DataFrame(tensor) # 변경이 잘 되었나 확인하기 위해 무작위로 50여개 체크. 데이터 프레임을 만들어 보기 좋게 함.

df_tensor[300:350]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
300,2,5,292,106,63,3,0,0,0,0,0,0,0,0,0
301,2,31,2836,15,36,3,0,0,0,0,0,0,0,0,0
302,2,269,1421,151,3,0,0,0,0,0,0,0,0,0,0
303,2,101,36,21,584,8,24,3,0,0,0,0,0,0,0
304,2,55,4,3474,39,75,3,0,0,0,0,0,0,0,0
305,2,24,3025,196,3,0,0,0,0,0,0,0,0,0,0
306,2,4,761,4216,3,0,0,0,0,0,0,0,0,0,0
307,2,44,75,24,629,196,3,0,0,0,0,0,0,0,0
308,2,625,7,1281,3,0,0,0,0,0,0,0,0,0,0
309,2,7,625,196,3,0,0,0,0,0,0,0,0,0,0


In [9]:
for idx in tokenizer.index_word:  # 토큰화 결과 및 해당 인덱스 예시 체크.
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : the
5 : i
6 : you
7 : and
8 : to
9 : a
10 : my


In [10]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장 생성
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성.
tgt_input = tensor[:, 1:]

print(src_input[150])  # 무작위로 결과 확인 
print(tgt_input[150])  # 무작위로 결과 확인

[   2 1348   17 1012   45  588   28    6 1024 2111    3    0    0    0]
[1348   17 1012   45  588   28    6 1024 2111    3    0    0    0    0]


## 4. 평가 데이터셋 분리

In [11]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 10)


print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


## 5. 인공지능 만들기

In [12]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000여개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만들기

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder= True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

# 데이터셋에서 데이터 한 배치만 불러오기
for src_sample, tgt_sample in dataset.take(1): break
    
# 한 배치만 불러온 데이터를 모델에 넣기
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.68788887e-04, -1.51679662e-04, -2.22644187e-04, ...,
          2.08989120e-04, -7.60233612e-04, -1.05638619e-05],
        [ 3.09371477e-04,  2.15698805e-04, -4.68449900e-04, ...,
         -5.20964750e-05, -9.55502968e-04,  7.84732219e-06],
        [ 5.84598922e-04,  6.35737379e-04, -3.77755583e-04, ...,
         -2.62707559e-04, -1.07139570e-03,  1.87582555e-04],
        ...,
        [ 1.24348141e-03,  1.85494311e-03, -2.65458552e-03, ...,
          4.47822706e-04,  5.70948527e-04,  1.22130488e-03],
        [ 1.45178754e-03,  1.91783579e-03, -3.15737561e-03, ...,
          5.06392753e-05,  5.91389253e-04,  8.91632226e-04],
        [ 8.50377313e-04,  1.78515119e-03, -3.01740598e-03, ...,
          3.55058786e-04,  3.45072069e-04,  5.22328657e-04]],

       [[ 1.68788887e-04, -1.51679662e-04, -2.22644187e-04, ...,
          2.08989120e-04, -7.60233612e-04, -1.05638619e-05],
        [ 5.08246558e-05,  1.77267764e-04, -4

In [14]:
model.summary() # 모델 생성 확인.

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  12289024  
_________________________________________________________________
lstm (LSTM)                  multiple                  8392704   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 41,375,457
Trainable params: 41,375,457
Non-trainable params: 0
_________________________________________________________________


In [15]:
#Loss
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
model.compile(loss=loss, optimizer = optimizer)
model.fit(dataset, epochs = 10)

Epoch 1/10
549/549 [==============================] - 105s 187ms/step - loss: 3.5447
Epoch 2/10
549/549 [==============================] - 103s 188ms/step - loss: 3.0951
Epoch 3/10
549/549 [==============================] - 103s 187ms/step - loss: 2.8908
Epoch 4/10
549/549 [==============================] - 103s 187ms/step - loss: 2.7317
Epoch 5/10
549/549 [==============================] - 103s 188ms/step - loss: 2.5931
Epoch 6/10
549/549 [==============================] - 103s 188ms/step - loss: 2.4662
Epoch 7/10
549/549 [==============================] - 103s 188ms/step - loss: 2.3478
Epoch 8/10
549/549 [==============================] - 103s 188ms/step - loss: 2.2356
Epoch 9/10
549/549 [==============================] - 103s 188ms/step - loss: 2.1281
Epoch 10/10
549/549 [==============================] - 103s 188ms/step - loss: 2.0252


In [16]:
def generate_text(model, tokenizer, init_sentence = "<start>", max_len=15):
    # 테스트를 위해 입력받은 init_sentence도 텐서 변환.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype = tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 단어 하나씩 예측해 문장 만들기.
    while True:
        # 1. 입력받은 문장의 텐서를 입력.
        predict = model(test_tensor)
        
        # 2. 예측된 값 중 가장 높은 확률인 word index 뽑아내기
        predict_word = tf.argmax(tf.nn.softmax(predict, axis = -1), axis = -1)[:, -1]
        
        # 3. 2에서 예측된 word index를 문장 뒤에 붙임.
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis = 0)], axis = -1)
        
        # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
            
    
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환함.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

    

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love you <end> '

# 1st try
## Action
### (1) 후렴 구문(chrous, repeat, tag) 표시 지움.
#### Source/Target train number: 140599
#### Embedding size: 256
#### Hidden size: 1024
### Loss at 10th epoch: 2.7608
### Sentence: ' < start > i love you so much, i love you < end > '

# 2nd
## Action
### (1) 중복구문 지우기, 
### (2) 한 줄에 두 문장이면 '.', '?', '!' 일 때 뒤에 < end > 붙여서 문장 종결
#### Source/Target train number: 93357
#### Embedding size: 256
#### Hidden size: 1024
### Loss at 10th epoch: 2.7608
### Sentence: ' < start > i love you so much, i love you < end > '

# 3rd
## Action
### (1) Embedding size 256 -> 512
#### Source/Target train number: 93357
#### Embedding size: 512
#### Hidden size: 1024
### Loss at 10th epoch: 2.6986
### Sentence: ' < start > i love you so much, i love you < end > '

# 4th
## Action
### (1) 중복구문 삭제 방식 변경: 비어있는 리스트를 만들고 그 안에 중복되지 않는 구문만 넣는 방식 -> 집합화해서 중복 제거 후 다시 리스트화.
#### Source/Target train number: 93357
#### Embedding size: 512
#### Hidden size: 1024
### Loss at 10th epoch: 2.7244
### Sentence: '< start > i love you < end > '

# 5th
## Action
### (1) tensor 마지막 열의 마감처리를 제대로 한 것으로 생각하고 있었는데, 되지 않았음.
### (2) 마지막 열에 대한 접근 방법을 찾고자 이것저것 시도해 보았으나 되지 않음.
### (3) 검색을 정확히 하려면 tensor type을 먼저 알아야겠다 싶어서 자료형을 조회 -> ndarray임을 확인.
### (4) 관련하여 검색, np.where에서 마지막 열에 조건을 달아 변경. 0이상의 숫자는 모두 3으로 치환하여 종결.
#### Source/Target train number: 93357
#### Embedding size: 512
#### Hidden size: 1024
### Loss at 10th epoch: 2.6587
### Sentence: '< start > i love you so much < end > '

# 6th
## Action
### (1) 중복 삭제를 취소함. 왜 더 높게 나오는가? 노래에선 반복문이 큰 문제가 되는 것은 아니라는 말이다.
#### Source/Target train number: 140599
#### Embedding size: 512
#### Hidden size: 1024
### Loss at 10th epoch: 2.2144
### Sentence: '< start > i love you so much, i love you < end > '

# 7th
## Action
### (1) Embedding size 512 -> 1024
#### Source/Target train number: 140599
#### Embedding size: 1024
#### Hidden size: 1024
## __Loss at 10th epoch: 2.0379__
## __Sentence: '< start > i love you so much, i love you so much, i love you < end > ' __

# 8th
## Action
### (1) Restart and run all
#### Source/Target train number: 140599
#### Embedding size: 1024
#### Hidden size: 1024
## __Loss at 10th epoch: 2.0252__
## __Sentence: ''< start > i love you < end > ' __

# 회고

### 초반에 텐서에서 기본 단위를 문장이 아닌 단어로 착각하여 source dataset과 target dataset을 이런 방식으로 설정하는 이유를 이해하지 못하였다. 하지만, 조원 및 퍼실분들과 여러 번 이야기 하여 다음 단어가 무엇이 올지를 학습하여 확률화 한다는 것을 이해하고 진행하였다.
### 문장 정제를 위해 re.sub 및 re.split에 대해 많은 공부를 하였다. 한 줄에 두 문장이 있는 것을 나누어 뒷 문장을 다음 행으로 추가시키고 싶었으나, 그러기엔 실력이 모자랐다. 많은 시간을 투자하였으나 포기.
### 문장 반복은 없애는 것이 손실을 줄일 수 있다고 생각해서 이 방법을 찾는데 많은 시간을 들였다. for 문을 사용하여 새 빈 리스트를 생성해서 여기에 미중복 행만 넣는 방식으로 만들었으나, 시간이 많이 걸렸다. 나중에 set 사용법을 알고 사용해 보니 훨씬 간단하고 시간도 절약되었다. 
### 이 고생에도 불구하고, 반복문 삭제는 손실을 줄이기는 커녕 늘리는 효과가 있어 매우 허무함을 느꼈다. 생각해보니, 반복문 자체는 어색한 것이 없다. 많이 등장하여 강조의 효과가 된다는 느낌. 
### 이에 더해 임베딩 크기를 늘리자 손실이 더 줄어들고, i love you를 반복하고 끝나는데.. 문장이 더 길어진 건 좋았으나 결국 같은 단어 반복이라 한계가 있음을 확인했다. 문장생성 함수만 수 차례 반복했는데도 동일했다. 노래란 것이 대부분 사랑에 대한 것이니 그럴 수 밖에 없지 않을까. 다양한 문장 구사를 하려면 역시 다양한 소스를 줘야한다는 사실을 다시 한 번 확인했다.